# INBOX 
See all incoming messages here

In [12]:
from s3i import IdentityProvider, Broker, UserMessage, TokenType, Key
import time 
import base64
%store -r 

In [13]:
def callback(ch, method, properties, body):
    print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S",
                                   time.localtime()) + "]: A new message has been received")
    body_str = body.decode('utf8').replace("'", '"')  # convert bytes to str
    uMsg = UserMessage(msg_blob=body_str)

    """Decrypt the message and verify the signature
    """
    if uMsg.pgpMsg.is_encrypted:
        sec_key = Key(path_demo_dir="key", filename="test_sec.asc")
        #uMsg.decryptAndVerify(pgpKey_sec, os.path.join(
        #        fp, "gpg"), s3i_directory)
        uMsg.decrypt(sec_key.key)
        uMsg.convertPgpToMsg()

    print("[S3I]: Subject of the message: " + uMsg.msg["subject"])
    print("[S3I]: Text of the message:  " + uMsg.msg["text"])

    attachments_list = uMsg.msg["attachments"]
    """
    store the attachment file in specified path
    """
    for attachment in attachments_list:
        with open("received_data/"+attachment["filename"], 'wb') as file:
            decode = base64.b64decode(attachment["data"])
            file.write(decode)
            print("[S3I]: Attachment " + attachment["filename"]
               + " of the message is stored in received_data")
        


In [14]:
username= "KWH-Team"
password= "V4yuJ31izh0GCH36954O"
s3i_identity_provider = IdentityProvider(grant_type='password', identity_provider_url="https://idp.s3i.vswf.dev/", realm='KWH',
                                             client_id="s3i:0c253262-428e-44be-a11a-b83566bd1f68", client_secret="8d9ad853-2bfb-44f1-ad47-bc2a6bad8a03", username=username, password=password)
access_token = s3i_identity_provider.get_token(TokenType.ACCESS_TOKEN)
#access_token = idp.get_token(TokenType.ACCESS_TOKEN)
broker = Broker(auth_form='Username/Password', username=" ", password=access_token, host="rabbitmq.s3i.vswf.dev")
broker.receive("s3ibs://"+"s3i:bf5a3b54-1ecc-4195-bc55-cb4757b895d0", callback)


[S3I][2020-05-18 20:36:24]: A new message has been received
[S3I][2020-05-18 20:36:24]: Decryption successful
[S3I]: Subject of the message: Kannik
[S3I]: Text of the message:  Hase2
s3i:8f9eb115-c2fa-4aa8-b411-ea962e931c29
VGhpcyBpcyBhbiBhdHRhY2htZW50
[S3I]: Attachment text.txt of the message is stored in received_data
[S3I][2020-05-18 20:52:32]: A new message has been received
[S3I][2020-05-18 20:52:32]: Decryption successful
[S3I]: Subject of the message: Kannik
[S3I]: Text of the message:  Hase2
s3i:8f9eb115-c2fa-4aa8-b411-ea962e931c29
VGhpcyBpcyBhbiBhdHRhY2htZW50
[S3I]: Attachment text.txt of the message is stored in received_data


KeyboardInterrupt: 